# Import file

In [1]:
import pandas as pd
data_path = '/home/jeanchristophe/dataFolder/ppty_goldset/goldset_dedup_grouped.csv'
gold_set = pd.read_csv(data_path)
data_path = '/home/jeanchristophe/dataFolder/ppty_goldset/mapping_id_tag.xlsx'
tag_id_mapping = pd.read_excel(data_path)

/home/jeanchristophe/.pyenv/versions/3.7.7/envs/doublons3.7/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/jeanchristophe/.pyenv/versions/3.7.7/envs/doublons3.7/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
gold_set

,idannonce,cp,ville,codeinsee,descriptif_fr,px,surface,annee_construct,nb_pieces,latitude,...,districts_name,region_id,min_px,max_px,min_surface,max_surface,min_nbpiece,max_nbpiece,groups,len_group
0,152917265,38100,Grenoble,380185.0,grenoble allee de la pelouse en bordure du par...,88000.0,80.08,NaN,4.0,45.162621,...,Arlequin,252,85360.0,90640.0,78.4784,81.6816,3.0,5.0,"[137918345, 152656465, 154105923]",3
1,151210099,38100,Grenoble,380185.0,era immobilier gresivaudan a trouve pour vous ...,158970.0,70.00,1960.0,3.0,45.180660,...,Allies-Clos d'Or,252,154200.9,163739.1,68.6000,71.4000,2.0,4.0,[142827435],1
2,150907823,38130,Échirolles,380151.0,iad france sophie armand vous propose echiroll...,125000.0,63.00,1970.0,4.0,45.147226,...,Comboire,252,121250.0,128750.0,61.7400,64.2600,3.0,5.0,"[151785547, 152595879, 154223735]",3
3,154693437,38950,Saint-Martin-le-Vinoux,380423.0,iad france diane cohade vous propose exclusivi...,135000.0,68.00,NaN,4.0,45.216072,...,Nord Est,252,130950.0,139050.0,66.6400,69.3600,3.0,5.0,"[143129747, 150783367, 145607233, 153329461, 1...",9
4,153731933,38000,Grenoble,380185.0,iad france colette bonnemaison vous propose gr...,228000.0,80.00,1949.0,3.0,45.179022,...,Abry,252,221160.0,234840.0,78.4000,81.6000,2.0,4.0,"[151131015, 154210349]",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,148296771,93300,Aubervilliers,930001.0,a min de metro porte la villette immeuble secu...,265000.0,74.00,NaN,4.0,NaN,...,NaN,239,255725.0,274275.0,71.7800,76.2200,3.0,5.0,"[152471347, 154517705, 152241143, 152306431, 1...",5
2047,152968579,75017,Paris 17ème,750117.0,marche lebon grand pieces renove de m dans imm...,667800.0,53.00,1900.0,2.0,NaN,...,NaN,239,644427.0,691173.0,51.4100,54.5900,1.0,3.0,"[154419047, 145808105]",2
2048,145561525,93400,St Ouen,930070.0,laforet saint ouen dans le quartier recherche ...,525000.0,86.80,NaN,4.0,NaN,...,NaN,239,506625.0,543375.0,84.1960,89.4040,3.0,5.0,"[145549993, 153617397, 154032987]",3
2049,154196547,93400,Saint-Ouen,930070.0,frais de notaire reduits au coeur de la ville ...,574000.0,86.00,NaN,4.0,NaN,...,NaN,239,553910.0,594090.0,83.4200,88.5800,3.0,5.0,"[154687809, 154195899, 154196199, 154195963]",4


In [3]:
import requests
import numpy as np
import copy

def tag_to_cdn(photo_tag, width = 300):
    if isinstance(photo_tag, str):
        url = f"https://v.seloger.com/s/width/{width}/visuels/{'/'.join(photo_tag[:4])}/{photo_tag}.jpg"
        return url
        
def id_to_tag(ads_id, mapping):
    return mapping[mapping.idannonce==ads_id]['tagphoto'].values.tolist()

def id_to_cdn(ads_id, mapping):
    cdn_list = []
    for tag in id_to_tag(ads_id, mapping):
        cdn_list.append(tag_to_cdn(tag))
    return cdn_list

def check_jsonl(line):
    # Get list of image-url
    if len(line['images'])==0:
        return False
    try:
        option_list = copy.deepcopy(line['options'])
    except:
        return False
    
    for opt in line['options']:
        if len(opt['images'])==0:
            option_list.remove(opt)
    if len(option_list)==0:
        return False
    else:
        line['options'] = option_list
        return line
    

In [4]:
json_line_list = []
for ind, row in gold_set.iterrows():
    
    line = {
        "id": str(row.idannonce),
        "text": row.descriptif_fr,
        "images": id_to_cdn(row.idannonce, tag_id_mapping)
    }

    
    options_list = []
    
    for id_linked in eval(row.groups):
        
        try:
            text_descriptif = gold_set[gold_set.idannonce==id_linked].descriptif_fr.values[0]
            
        except:
            print('error with idannonce', line['id'], 'and id_linked', id_linked, 'and indice', ind)
            
        else:
            options_dict = {
                "id": id_linked,
                "text": text_descriptif,
                "images": id_to_cdn(id_linked, tag_id_mapping)
            }

        finally:
            options_list.append(options_dict)
            

    line["options"] = options_list

        
    if check_jsonl(line):
        json_line_list.append(line)

    print(f'{ind+1}/{gold_set.shape[0]}')
        

1/2051
2/2051
3/2051
4/2051
5/2051
6/2051
7/2051
8/2051
9/2051
10/2051
11/2051
12/2051
13/2051
14/2051
15/2051
16/2051
17/2051
18/2051
19/2051
20/2051
21/2051
22/2051
23/2051
24/2051
25/2051
26/2051
27/2051
28/2051
29/2051
30/2051
31/2051
32/2051
33/2051
34/2051
35/2051
36/2051
37/2051
38/2051
39/2051
40/2051
41/2051
42/2051
43/2051
44/2051
45/2051
46/2051
47/2051
48/2051
49/2051
50/2051
51/2051
52/2051
53/2051
54/2051
55/2051
56/2051
57/2051
58/2051
59/2051
60/2051
61/2051
62/2051
63/2051
64/2051
65/2051
66/2051
67/2051
68/2051
69/2051
70/2051
71/2051
72/2051
73/2051
74/2051
75/2051
76/2051
77/2051
78/2051
79/2051
80/2051
81/2051
82/2051
83/2051
84/2051
85/2051
86/2051
87/2051
88/2051
89/2051
90/2051
91/2051
92/2051
93/2051
94/2051
95/2051
96/2051
97/2051
98/2051
99/2051
100/2051
101/2051
102/2051
103/2051
104/2051
105/2051
106/2051
107/2051
108/2051
109/2051
110/2051
111/2051
112/2051
113/2051
114/2051
115/2051
116/2051
117/2051
118/2051
119/2051
120/2051
121/2051
122/2051
123/2051
1

948/2051
949/2051
950/2051
951/2051
952/2051
953/2051
954/2051
955/2051
956/2051
957/2051
958/2051
959/2051
960/2051
961/2051
962/2051
963/2051
964/2051
965/2051
966/2051
967/2051
968/2051
969/2051
970/2051
971/2051
972/2051
973/2051
974/2051
975/2051
976/2051
977/2051
978/2051
979/2051
980/2051
981/2051
982/2051
983/2051
984/2051
985/2051
986/2051
987/2051
988/2051
989/2051
990/2051
991/2051
992/2051
993/2051
994/2051
995/2051
996/2051
997/2051
998/2051
999/2051
1000/2051
1001/2051
1002/2051
1003/2051
1004/2051
1005/2051
1006/2051
1007/2051
1008/2051
1009/2051
1010/2051
1011/2051
1012/2051
1013/2051
1014/2051
1015/2051
1016/2051
1017/2051
1018/2051
1019/2051
1020/2051
1021/2051
1022/2051
1023/2051
1024/2051
1025/2051
1026/2051
1027/2051
1028/2051
1029/2051
1030/2051
1031/2051
1032/2051
1033/2051
1034/2051
1035/2051
1036/2051
1037/2051
1038/2051
1039/2051
1040/2051
1041/2051
1042/2051
1043/2051
1044/2051
1045/2051
1046/2051
1047/2051
1048/2051
1049/2051
1050/2051
1051/2051
1052/2051
10

1764/2051
1765/2051
1766/2051
1767/2051
1768/2051
1769/2051
1770/2051
1771/2051
1772/2051
1773/2051
1774/2051
1775/2051
1776/2051
1777/2051
1778/2051
1779/2051
1780/2051
1781/2051
1782/2051
1783/2051
1784/2051
1785/2051
1786/2051
1787/2051
1788/2051
1789/2051
1790/2051
1791/2051
1792/2051
1793/2051
1794/2051
1795/2051
1796/2051
1797/2051
1798/2051
1799/2051
1800/2051
1801/2051
1802/2051
1803/2051
1804/2051
1805/2051
1806/2051
1807/2051
1808/2051
1809/2051
1810/2051
1811/2051
1812/2051
1813/2051
1814/2051
1815/2051
1816/2051
1817/2051
1818/2051
1819/2051
1820/2051
1821/2051
1822/2051
1823/2051
1824/2051
1825/2051
1826/2051
1827/2051
1828/2051
1829/2051
1830/2051
1831/2051
1832/2051
1833/2051
1834/2051
1835/2051
1836/2051
1837/2051
1838/2051
1839/2051
1840/2051
1841/2051
1842/2051
1843/2051
1844/2051
1845/2051
1846/2051
1847/2051
1848/2051
1849/2051
1850/2051
1851/2051
1852/2051
1853/2051
1854/2051
1855/2051
1856/2051
1857/2051
1858/2051
1859/2051
1860/2051
1861/2051
1862/2051
1863/2051


In [5]:
import jsonlines

with jsonlines.open('/home/jeanchristophe/dataFolder/ppty_goldset/task.jsonl', mode='w') as writer:
    writer.write_all(json_line_list)

In [6]:
len(json_line_list)

990